# Healthcare Disparities Analysis Based on Demographics!

The goal is to examine how patient demographics (e.g. Age, Gender, Marital Status) influence treatment and healthcare access, identifying disparities in the quality of care. 

In [49]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Disparities in Healthcare Analysis").getOrCreate()
from pyspark.sql.functions import year, current_date, concat_ws, when, col, trim, lower,count, mean, stddev, to_timestamp, unix_timestamp, regexp_replace, sum
import boto3
import io
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Initial setting up Data Frames and Exploration

We have already completed the ETL portion of extracting the data from individual patient json files. The extraction was done based on categories such as observation, immunization, diagnostic  reports, enocounters, claims data etc. That data is stored in parquet format for each category and is being used here below to examine what each dataframe looks like at the moment.    

In [50]:
import sys

input = sys.argv[1]
s3 = boto3.client("s3")
bucket = sys.argv[2]
base_key = sys.argv[3]
#input = "etl"

observation_df = spark.read.parquet(f"{input}/observation")
immunization_df = spark.read.parquet(f"{input}/immunization")
diagnostic_report_df = spark.read.parquet(f"{input}/diagnostic_report")
procedure_df = spark.read.parquet(f"{input}/procedure")
careteam_df = spark.read.parquet(f"{input}/careteam")
careplan_df = spark.read.parquet(f"{input}/careplan")
explanation_of_benefit_df = spark.read.parquet(f"{input}/explanation_of_benefit")
claim_df = spark.read.parquet(f"{input}/claim")
medication_request_df = spark.read.parquet(f"{input}/medication_request")
condition_df = spark.read.parquet(f"{input}/condition")
encounter_df = spark.read.parquet(f"{input}/encounter")
patient_df = spark.read.parquet(f"{input}/patient")


#observation_df.show(5)
# immunization_df.show(5)
# diagnostic_report_df.show(5)
# procedure_df.show(5)
# careteam_df.show(5)
# careplan_df.show(5)
# explanation_of_benefit_df.show(5)
# claim_df.show(5)
# medication_request_df.show(5)
# condition_df.show(5)
# encounter_df.show(5)



We looked at all the dataframes and the one's that interest us the most are the patients, encounters and the explanation of benefit dataframe. We are going to start with the patient data frame as it contains the key variables on which the project is going to focus. 

In [51]:
patient_df.show(5)
patient_df.printSchema()

+--------------------+--------------------+-----------+----------+------+----------+--------------------+--------------------+-------+-----------+-----------------+-------------------+------------+--------------+--------------------+---------------+--------------------+------------------+------------------------------+---------------------------+--------------+---------+
|          patient_id|    identifier_types|  last_name|first_name|gender|birth_date|                city|               state|country|postal_code|         latitude|          longitude|       phone|marital_status| mothers_maiden_name|birthplace_city|    birthplace_state|birthplace_country|disability_adjusted_life_years|quality_adjusted_life_years|multiple_birth|languages|
+--------------------+--------------------+-----------+----------+------+----------+--------------------+--------------------+-------+-----------+-----------------+-------------------+------------+--------------+--------------------+---------------+---

In [52]:
cities_count = patient_df.select("city").distinct().count()
print(f"Number of cities: {cities_count}")
patient_df.select("city").distinct().show()
patient_df.select("state").distinct().show()

patient_df.select("languages").show()
patient_df.select("marital_status").distinct().show()
patient_df.select("gender").distinct().show()

Number of cities: 14
+--------------------+
|                city|
+--------------------+
|Division No. 4, S...|
|         Clarenville|
|           Woodstock|
|   Wallace Hills 14A|
|      North Cowichan|
|    Hay River Dene 1|
|        Powell River|
|            Montréal|
|         Clyde River|
|       Prince Albert|
|             Toronto|
|              Dieppe|
|             Chambly|
|              Dawson|
+--------------------+

+--------------------+
|               state|
+--------------------+
|Newfoundland and ...|
|         Nova Scotia|
|    British Columbia|
|Northwest Territo...|
|              Quebec|
|Prince Edward Island|
|        Saskatchewan|
|             Ontario|
|       New Brunswick|
|               Yukon|
+--------------------+

+---------+
|languages|
+---------+
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
|[English]|
+---------+

+-----------

After the initial exploration, the primary areas to focus on are Gender, Marital Status and Age. We expected to have socioeconomic and ethnic data in the demographics, but they were not present in the dataset. Age is calculated based on the birth year and the current date. We drop the columns that are not needed and store them in a new dataframe that is called patient_df_cleaned. 

In [53]:
patient_df_cleaned = patient_df.select(
    "patient_id",
    "gender",
    "birth_date",
    "state",
    "postal_code",
    "marital_status",
    "disability_adjusted_life_years",
    "quality_adjusted_life_years",
)

### Calculating age

In [54]:
patient_df_cleaned = patient_df_cleaned.withColumn(
    "age",
    year(current_date()) - year(col("birth_date"))
)

patient_df_cleaned.show()

+--------------------+------+----------+--------------------+-----------+--------------+------------------------------+---------------------------+---+
|          patient_id|gender|birth_date|               state|postal_code|marital_status|disability_adjusted_life_years|quality_adjusted_life_years|age|
+--------------------+------+----------+--------------------+-----------+--------------+------------------------------+---------------------------+---+
|37b0788d-2389-b70...|  male|1977-02-03|Newfoundland and ...|        A1A|             M|             20.28075539923084|          17.71924460076916| 47|
|7ca297d1-94e0-806...|female|1955-05-02|Newfoundland and ...|        A5A|             S|            2.7338625827639547|         54.266137417236045| 69|
|000002d9-3788-19b...|female|2019-05-13|         Nova Scotia|        B3K| Never Married|                           0.0|                        1.0|  5|
|047c5aaf-d8cb-a31...|female|1965-08-20|Newfoundland and ...|        A1A|             S|

### Standardizing Marital Status 

In [55]:

patient_df_cleaned = patient_df_cleaned.withColumn(
    "marital_status",
    trim(lower(col("marital_status")))  
)

patient_df_cleaned = patient_df_cleaned.withColumn(
    "marital_status",
    when(col("marital_status") == "m", "Married")
    .when(col("marital_status") == "s", "Single")
    .when(col("marital_status") == "never married", "Never Married")
    .otherwise("Unknown")  
)

patient_df_cleaned.select("marital_status").distinct().show()

+--------------+
|marital_status|
+--------------+
|       Married|
|        Single|
| Never Married|
+--------------+



Making sure there are no patients without a patient ID

In [56]:
null_patient_id = patient_df_cleaned.filter(col("patient_id").isNull()).count()

print(f"null 'patient_id': {null_patient_id}")

null 'patient_id': 0


### Observations from the Patient dataset. 

We look at the distribution of patients by gender. We calculate the average age of patients and the std deviation between them. Then we look at the marital status of the patients. 

In [57]:
patient_df_cleaned.groupBy("gender").agg(count("*").alias("count")).show()

+------+-----+
|gender|count|
+------+-----+
|  male|    7|
|female|    8|
+------+-----+



In [58]:
age_stats = patient_df_cleaned.select(
    mean("age").alias("average_age"),
    stddev("age").alias("stddev_age")
)
age_stats.show()

+-----------------+------------------+
|      average_age|        stddev_age|
+-----------------+------------------+
|54.06666666666667|28.704073047632274|
+-----------------+------------------+



In [59]:
patient_df_cleaned.groupBy("marital_status").agg(count("*").alias("count")).show()

+--------------+-----+
|marital_status|count|
+--------------+-----+
|       Married|    5|
|        Single|    5|
| Never Married|    5|
+--------------+-----+



In [60]:
patient_df_cleaned.groupBy("gender", "marital_status").agg(
    mean("age").alias("average_age"),
    stddev("age").alias("stddev_age"),
    count("*").alias("count")
).show(truncate=False)

+------+--------------+------------------+------------------+-----+
|gender|marital_status|average_age       |stddev_age        |count|
+------+--------------+------------------+------------------+-----+
|male  |Married       |52.0              |7.0710678118654755|2    |
|female|Single        |64.0              |7.0710678118654755|2    |
|female|Never Married |16.333333333333332|10.598742063723098|3    |
|male  |Single        |67.33333333333333 |17.009801096230763|3    |
|male  |Never Married |28.5              |2.1213203435596424|2    |
|female|Married       |90.33333333333333 |13.576941236277534|3    |
+------+--------------+------------------+------------------+-----+



Uploading plots to S3 function

In [61]:
def upload_plot_to_s3(plt, plot_name):
    
    buffer = io.BytesIO()
    plt.savefig(buffer, format="png")
    buffer.seek(0)  
    
    key = f"{base_key}/{plot_name}.png"
    
    s3.put_object(Bucket=bucket, Key=key, Body=buffer, ContentType="image/png")
    
    buffer.close()
    
    print(f"Plot successfully uploaded to s3://{bucket}/{key}")

In [ ]:
gender_counts = patient_df_cleaned.groupBy("gender").count().toPandas()
age_counts = patient_df_cleaned.groupBy("age").count().toPandas()
marital_status_counts = patient_df_cleaned.groupBy("marital_status").count().toPandas()
age_by_gender_marital_status = patient_df_cleaned.groupBy("gender", "marital_status").agg(
    mean("age").alias("average_age"),
    stddev("age").alias("stddev_age"),
    count("*").alias("count")
).toPandas()

#### Gender Distribution Bar Plot

In [ ]:

plt.figure(figsize=(8, 6))
sns.barplot(data=gender_counts, x="gender", y="count", palette="Set2")
plt.title("Gender Distribution")
plt.xlabel("Gender")
plt.ylabel("Count")
upload_plot_to_s3(plt, "gender_distribution")

#### Age Distribution (Box Plot)

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x="age", data=age_counts, color="lightblue")
plt.title("Age Distribution")
plt.xlabel("Age")
upload_plot_to_s3(plt, "age_distribution_boxplot")

#### Marital Status Distribution Bar Plot

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(data=marital_status_counts, x="marital_status", y="count", palette="muted")
plt.title("Marital Status Distribution")
plt.xlabel("Marital Status")
plt.ylabel("Count")
upload_plot_to_s3(plt, "marital_status_distribution")

#### Average Age by Gender and Marital Status

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=age_by_gender_marital_status, x="gender", y="average_age", hue="marital_status")
plt.title("Average Age by Gender and Marital Status")
plt.xlabel("Gender")
plt.ylabel("Average Age")
upload_plot_to_s3(plt, "average_age_by_gender_marital_status")

# Encounters 

After examining the patient df and demographics, now we want to see what kind of healtchare access usually patients get. We start by examining and cleaning the df encounters.


In [62]:
encounter_df.printSchema()
encounter_df.show(10, truncate=False)

root
 |-- encounter_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- class_code: string (nullable = true)
 |-- type_text: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- patient_reference: string (nullable = true)
 |-- service_provider_id: string (nullable = true)
 |-- service_provider_display: string (nullable = true)
 |-- participant_individual_display: string (nullable = true)
 |-- participant_individual_reference: string (nullable = true)
 |-- participant_period_start: string (nullable = true)
 |-- participant_period_end: string (nullable = true)
 |-- participant_type_code: string (nullable = true)
 |-- participant_type_display: string (nullable = true)

+------------------------------------+--------+----------+------------------------------------------+-------------------------+-------------------------+---------------------------------------------+-----------------------------------------------

We look at differennt HL7 class codes and then standardize them based on the information we got from https://terminology.hl7.org/1.0.0/ValueSet-v3-ActEncounterCode.html 

In [63]:
encounter_df.select("class_code").distinct().show(truncate=False)

+----------+
|class_code|
+----------+
|IMP       |
|AMB       |
|EMER      |
+----------+



In [64]:
encounter_df.select("status").distinct().show(truncate=False)

+--------+
|status  |
+--------+
|finished|
+--------+



We create an cleaned encounter df and convert the timestamps from strings so that we can calculate the enocunter duration for each encounter 

In [65]:
encounter_df_cleaned = encounter_df
encounter_df_cleaned = encounter_df.dropna(subset=["encounter_id", "patient_reference", "start_time"])
encounter_df_cleaned = encounter_df_cleaned.withColumn("start_time", to_timestamp("start_time"))
encounter_df_cleaned = encounter_df_cleaned.withColumn("end_time", to_timestamp("end_time"))
encounter_df_cleaned = encounter_df_cleaned.withColumn(
    "encounter_duration",
    (unix_timestamp("end_time") - unix_timestamp("start_time")) / 60  
)

In [66]:
encounter_df_cleaned = encounter_df_cleaned.withColumn(
    "status",
    when(col("status") == "finished", "Completed")
    .otherwise("In-Progress")
)

In [67]:
encounter_df_cleaned.select("status").distinct().show() #Probably going to drop if only finished values

+---------+
|   status|
+---------+
|Completed|
+---------+



In [68]:
encounter_df_cleaned = encounter_df_cleaned.withColumn(
    "class_code",
    when(col("class_code") == "IMP", "Inpatient")
    .when(col("class_code") == "AMB", "Ambulatory")
    .when(col("class_code") == "EMER", "Emergency")
    .otherwise("Unknown")  
)

encounter_df_cleaned.select("class_code").distinct().show()

+----------+
|class_code|
+----------+
|Ambulatory|
| Emergency|
| Inpatient|
+----------+



The patient_reference is not in correct format so we have to strip the initial terms from it to match it with patient id's in patient df.

In [69]:
encounter_df_cleaned = encounter_df_cleaned.withColumn(
    "patient_reference",
    regexp_replace("patient_reference", "urn:uuid:", "")
)

encounter_df_cleaned.select("patient_reference").distinct().show(truncate=False)
encounter_df_cleaned.show(10, truncate=False)

+------------------------------------+
|patient_reference                   |
+------------------------------------+
|7ca297d1-94e0-8069-4cdb-50f313de9a5f|
|b4b67625-c44c-a832-7aba-3a2ae02a4ef1|
|0ff36c90-7124-3569-9591-75c6386f45f5|
|c8af2e36-e7d2-fc26-de8c-8936c9e4705e|
|37b0788d-2389-b706-52e0-7938f80f3f23|
|3ed96c8a-2671-2959-57cd-b3aeb8b16af0|
|43dceade-79e7-fc0f-8245-59ec5feb4174|
|8fc4d8c6-1ae5-2736-1158-ef55c405db8d|
|5bbda3f6-ee30-6a51-ebbd-19cd0a6b119d|
|33c6c178-b195-c16c-0045-02f5f4b53073|
|d59c9ad1-8a83-f0f0-a080-635ff42b43fa|
|fb47aa81-57a0-edc1-185d-891ad133034b|
|fbcfe59a-ee76-74ed-240c-a0f3c9d46c04|
|000002d9-3788-19b0-0631-c4cad43fe513|
|047c5aaf-d8cb-a31b-189a-f28e0efb78e0|
+------------------------------------+

+------------------------------------+---------+----------+------------------------------------------+-------------------+-------------------+------------------------------------+-------------------------------------------------------------------------------

In [70]:
encounter_df_cleaned.select("patient_reference").distinct().count()
encounter_df_cleaned.select("patient_reference").distinct().show(truncate=False)

+------------------------------------+
|patient_reference                   |
+------------------------------------+
|7ca297d1-94e0-8069-4cdb-50f313de9a5f|
|b4b67625-c44c-a832-7aba-3a2ae02a4ef1|
|0ff36c90-7124-3569-9591-75c6386f45f5|
|c8af2e36-e7d2-fc26-de8c-8936c9e4705e|
|37b0788d-2389-b706-52e0-7938f80f3f23|
|3ed96c8a-2671-2959-57cd-b3aeb8b16af0|
|43dceade-79e7-fc0f-8245-59ec5feb4174|
|8fc4d8c6-1ae5-2736-1158-ef55c405db8d|
|5bbda3f6-ee30-6a51-ebbd-19cd0a6b119d|
|33c6c178-b195-c16c-0045-02f5f4b53073|
|d59c9ad1-8a83-f0f0-a080-635ff42b43fa|
|fb47aa81-57a0-edc1-185d-891ad133034b|
|fbcfe59a-ee76-74ed-240c-a0f3c9d46c04|
|000002d9-3788-19b0-0631-c4cad43fe513|
|047c5aaf-d8cb-a31b-189a-f28e0efb78e0|
+------------------------------------+



We join the cleaned encounter df with cleaned patient df.

In [71]:
encounter_patient_df = encounter_df_cleaned.join(
    patient_df_cleaned,
    encounter_df_cleaned.patient_reference == patient_df_cleaned.patient_id,
    how="inner"
)

In [72]:
encounter_patient_df.printSchema()
encounter_patient_df.show(10, truncate=False)

root
 |-- encounter_id: string (nullable = true)
 |-- status: string (nullable = false)
 |-- class_code: string (nullable = false)
 |-- type_text: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- patient_reference: string (nullable = true)
 |-- service_provider_id: string (nullable = true)
 |-- service_provider_display: string (nullable = true)
 |-- participant_individual_display: string (nullable = true)
 |-- participant_individual_reference: string (nullable = true)
 |-- participant_period_start: string (nullable = true)
 |-- participant_period_end: string (nullable = true)
 |-- participant_type_code: string (nullable = true)
 |-- participant_type_display: string (nullable = true)
 |-- encounter_duration: double (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (null

Dropping the unnecessary columns from the encounter_patient_df

In [73]:
encounter_patient_df = encounter_patient_df.drop(
    "service_provider_id",
    "service_provider_display",
    "participant_individual_display",
    "participant_individual_reference",
    "participant_period_start",
    "participant_period_end",
    "participant_type_code",
    "participant_type_display"
)

In [74]:
overlap_count = encounter_df_cleaned.join(
    patient_df_cleaned,
    encounter_df_cleaned.patient_reference == patient_df_cleaned.patient_id,
    how="inner"
).count()
print(f"Number of overlapping records: {overlap_count}")

Number of overlapping records: 887


In [75]:
encounter_patient_df.groupBy("gender", "class_code").count().show()

+------+----------+-----+
|gender|class_code|count|
+------+----------+-----+
|female| Inpatient|   33|
|female| Emergency|   24|
|female|Ambulatory|  479|
|  male|Ambulatory|  333|
|  male| Emergency|    9|
|  male| Inpatient|    9|
+------+----------+-----+



#### Gender vs Class Code Count

In [ ]:
gender_class_code_counts = encounter_patient_df.groupBy("gender", "class_code").count().toPandas()

pivot_data = gender_class_code_counts.pivot(index="gender", columns="class_code", values="count").fillna(0)


pivot_data.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="Set2")
plt.title("Gender vs Class Code Encounter Distribution")
plt.xlabel("Gender")
plt.ylabel("Count")
plt.legend(title="Class Code")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "Gender_vs_Class_Code_Count")

In [76]:
encounter_patient_df.groupBy("gender").agg(mean("encounter_duration").alias("avg_duration")).show()

+------+-----------------+
|gender|     avg_duration|
+------+-----------------+
|female| 76265.7761504975|
|  male|90.67079772079772|
+------+-----------------+



#### Gender vs Average Encounter Duration Plot

In [ ]:
gender_avg_duration = encounter_patient_df.groupBy("gender").agg(mean("encounter_duration").alias("avg_duration")).toPandas()

plt.figure(figsize=(8, 6))
sns.barplot(data=gender_avg_duration, x="gender", y="avg_duration", palette="Set2")
plt.title("Average Encounter Duration by Gender")
plt.xlabel("Gender")
plt.ylabel("Average Duration")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "Gender_vs_Avg_Encounter_Duration")

Creating Age groups to use ages as a classifier in our analysis 

In [77]:
from pyspark.sql.functions import when

encounter_patient_df = encounter_patient_df.withColumn(
    "age_group",
    when(col("age") < 18, "Child")
    .when((col("age") >= 18) & (col("age") < 40), "Young Adult")
    .when((col("age") >= 40) & (col("age") < 65), "Middle Aged")
    .otherwise("Senior")
)

encounter_patient_df.groupBy("age_group").agg(mean("encounter_duration").alias("avg_duration")).show()


+-----------+------------------+
|  age_group|      avg_duration|
+-----------+------------------+
|     Senior| 78647.56153846154|
|Middle Aged| 45.78261802575108|
|Young Adult|21.976933333333335|
|      Child|              15.0|
+-----------+------------------+



#### Age Group vs Average Encounter Duration Plot

In [ ]:
age_group_avg_duration = encounter_patient_df.groupBy("age_group").agg(mean("encounter_duration").alias("avg_duration")).toPandas()

plt.figure(figsize=(8, 6))
sns.lineplot(data=age_group_avg_duration, x="age_group", y="avg_duration", marker="o", palette="Set2")
plt.title("Average Encounter Duration by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Average Duration")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "AgeGroup_vs_Average_Encounter_Duration")


In [41]:
encounter_patient_df.orderBy(col("encounter_duration").desc()).show(10, truncate=False)


+------------------------------------+---------+----------+--------------------------------------------+-------------------+-------------------+------------------------------------+------------------+------------------------------------+------+----------+-------------------------+-----------+--------------+------------------------------+---------------------------+---+---------+
|encounter_id                        |status   |class_code|type_text                                   |start_time         |end_time           |patient_reference                   |encounter_duration|patient_id                          |gender|birth_date|state                    |postal_code|marital_status|disability_adjusted_life_years|quality_adjusted_life_years|age|age_group|
+------------------------------------+---------+----------+--------------------------------------------+-------------------+-------------------+------------------------------------+------------------+------------------------------------

Some patients may be in the hospital for a significant long period of time, we are trying to remove these outliers from our dataset. 

In [42]:

inpatient_long_stay = encounter_patient_df.filter((col("class_code") == "Inpatient") & (col("encounter_duration") > 1440))


inpatient_long_stay.orderBy(col("encounter_duration").desc()).show(10, truncate=False)


+------------------------------------+---------+----------+--------------------------------------------+-------------------+-------------------+------------------------------------+------------------+------------------------------------+------+----------+-------------------------+-----------+--------------+------------------------------+---------------------------+---+---------+
|encounter_id                        |status   |class_code|type_text                                   |start_time         |end_time           |patient_reference                   |encounter_duration|patient_id                          |gender|birth_date|state                    |postal_code|marital_status|disability_adjusted_life_years|quality_adjusted_life_years|age|age_group|
+------------------------------------+---------+----------+--------------------------------------------+-------------------+-------------------+------------------------------------+------------------+------------------------------------

Finding out avg and max duration of different types of encounters 

In [43]:

from pyspark.sql.functions import max

encounter_patient_df.groupBy("class_code").agg(
    mean("encounter_duration").alias("avg_duration"),
    max("encounter_duration").alias("max_duration")
).show()



+----------+------------------+-----------------+
|class_code|      avg_duration|     max_duration|
+----------+------------------+-----------------+
|Ambulatory|18.077073070607554|99.93333333333334|
| Emergency| 382.1515151515151|          10620.0|
| Inpatient| 973404.5686507936|       4.079088E7|
+----------+------------------+-----------------+



In [44]:
filtered_encounter_patient_df = encounter_patient_df.filter(
    (col("encounter_duration") <= 86400) | (col("class_code") != "Inpatient")
)

filtered_encounter_patient_df.show(10, truncate=False)

#86400 is equal to 60 days in the hospital 

+------------------------------------+---------+----------+------------------------------------------+-------------------+-------------------+------------------------------------+------------------+------------------------------------+------+----------+-------------------------+-----------+--------------+------------------------------+---------------------------+---+---------+
|encounter_id                        |status   |class_code|type_text                                 |start_time         |end_time           |patient_reference                   |encounter_duration|patient_id                          |gender|birth_date|state                    |postal_code|marital_status|disability_adjusted_life_years|quality_adjusted_life_years|age|age_group|
+------------------------------------+---------+----------+------------------------------------------+-------------------+-------------------+------------------------------------+------------------+------------------------------------+-----

Creating a seperate df for outliers

In [46]:
outliers_df = encounter_patient_df.filter(col("encounter_duration") > 86400)
outliers_df.show()

+--------------------+---------+----------+--------------------+-------------------+-------------------+--------------------+------------------+--------------------+------+----------+-------------+-----------+--------------+------------------------------+---------------------------+---+---------+
|        encounter_id|   status|class_code|           type_text|         start_time|           end_time|   patient_reference|encounter_duration|          patient_id|gender|birth_date|        state|postal_code|marital_status|disability_adjusted_life_years|quality_adjusted_life_years|age|age_group|
+--------------------+---------+----------+--------------------+-------------------+-------------------+--------------------+------------------+--------------------+------+----------+-------------+-----------+--------------+------------------------------+---------------------------+---+---------+
|84006e4b-e560-32c...|Completed| Inpatient|Encounter for sym...|1941-11-23 19:28:00|2019-06-14 20:28:00|b4

In [47]:
filtered_encounter_patient_df.groupBy("class_code").agg(
    mean("encounter_duration").alias("avg_duration")
).show()


+----------+------------------+
|class_code|      avg_duration|
+----------+------------------+
|Ambulatory|18.077073070607554|
| Emergency| 382.1515151515151|
| Inpatient|2246.6313008130082|
+----------+------------------+



#### Class Code vs Average Encounter Duration with Filtered Data Plot

In [ ]:
class_code_duration_stats = encounter_patient_df.groupBy("class_code").agg(
    mean("encounter_duration").alias("avg_duration"),
    max("encounter_duration").alias("max_duration")
).toPandas()

plt.figure(figsize=(10, 6))
sns.scatterplot(data=class_code_duration_stats, x="avg_duration", y="max_duration", hue="class_code", palette="Set2", s=100)
plt.title("Average vs Max Encounter Duration by Class Code")
plt.xlabel("Average Duration")
plt.ylabel("Max Duration")
plt.legend(title="Class Code")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "Class_Code_vs_Average_Encounter_Duration_with_Filtered_Data")

In [48]:
# Group by age_group and class_code with filtered data
filtered_encounter_patient_df.groupBy("class_code", "age_group").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("class_code", "age_group").show()


+----------+-----------+------------------+
|class_code|  age_group|      avg_duration|
+----------+-----------+------------------+
|Ambulatory|      Child|              15.0|
|Ambulatory|Middle Aged| 20.39170403587444|
|Ambulatory|     Senior|16.520860448300798|
|Ambulatory|Young Adult|20.000980392156865|
| Emergency|Middle Aged|              60.0|
| Emergency|     Senior| 565.9047619047619|
| Emergency|Young Adult|61.166666666666664|
| Inpatient|Middle Aged|            1440.0|
| Inpatient|     Senior| 2333.834684684685|
+----------+-----------+------------------+



#### Heatmap for Class Code, Age Group vs Average Encounter Duration

In [ ]:
class_code_age_group_avg_duration = filtered_encounter_patient_df.groupBy("class_code", "age_group").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("class_code", "age_group").toPandas()

pivot_data = class_code_age_group_avg_duration.pivot(
    index="class_code",  
    columns="age_group",  
    values="avg_duration"  
).fillna(0)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_data, annot=True, cmap="coolwarm", fmt=".1f")
plt.title("Average Encounter Duration by Class Code and Age Group (Heatmap)")
plt.xlabel("Age Group")
plt.ylabel("Class Code")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "Heatmap_for_Class_Code_Age_Group_Average_Encounter")

In [78]:
# Group by class_code, age_group, and gender to calculate average duration
filtered_encounter_patient_df.groupBy("class_code", "age_group", "gender").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("class_code", "age_group", "gender").show(truncate=False)


+----------+-----------+------+------------------+
|class_code|age_group  |gender|avg_duration      |
+----------+-----------+------+------------------+
|Ambulatory|Child      |female|15.0              |
|Ambulatory|Middle Aged|female|15.0              |
|Ambulatory|Middle Aged|male  |20.618457943925236|
|Ambulatory|Senior     |female|16.398496905393458|
|Ambulatory|Senior     |male  |17.070039682539683|
|Ambulatory|Young Adult|female|21.500198412698417|
|Ambulatory|Young Adult|male  |16.402857142857144|
|Emergency |Middle Aged|female|60.0              |
|Emergency |Middle Aged|male  |60.0              |
|Emergency |Senior     |female|616.7894736842105 |
|Emergency |Senior     |male  |82.5              |
|Emergency |Young Adult|female|61.75             |
|Emergency |Young Adult|male  |60.0              |
|Inpatient |Middle Aged|male  |1440.0            |
|Inpatient |Senior     |female|2102.867708333333 |
|Inpatient |Senior     |male  |3812.0233333333335|
+----------+-----------+------+

#### Heatmap for Class Code, Age Group, Gender vs Average Encounter Duration

In [ ]:
class_code_age_group_gender_avg_duration = filtered_encounter_patient_df.groupBy("class_code", "age_group", "gender").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("class_code", "age_group", "gender").toPandas()

pivot_data = class_code_age_group_gender_avg_duration.pivot_table(index=["class_code", "age_group"], columns="gender", values="avg_duration").fillna(0)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_data, annot=True, cmap="coolwarm", fmt=".1f")
plt.title("Average Encounter Duration by Class Code, Age Group, and Gender (Heatmap)")
plt.xlabel("Gender")
plt.ylabel("Class Code and Age Group")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "Heatmap_for_Class_Code_Age_Group_Gender_Average_Encounter")

In [79]:
filtered_encounter_patient_df.groupBy("marital_status").agg(
    mean("encounter_duration").alias("avg_duration")
).show()

+--------------+------------------+
|marital_status|      avg_duration|
+--------------+------------------+
|        Single| 89.80153631284917|
|       Married|214.13807106598986|
| Never Married|21.508333333333333|
+--------------+------------------+



In [80]:
filtered_encounter_patient_df.groupBy("marital_status", "gender").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("marital_status", "gender").show()

+--------------+------+------------------+
|marital_status|gender|      avg_duration|
+--------------+------+------------------+
|       Married|female| 288.8011320754717|
|       Married|  male| 60.76046511627907|
| Never Married|female|22.556872852233678|
| Never Married|  male| 18.75945945945946|
|        Single|female| 51.18901734104047|
|        Single|  male|125.90945945945946|
+--------------+------+------------------+



#### Marital Status vs Gender vs Average Encounter Duration

In [ ]:
marital_gender_avg_duration = filtered_encounter_patient_df.groupBy("marital_status", "gender").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("marital_status", "gender").toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=marital_gender_avg_duration, x="marital_status", y="avg_duration", hue="gender", palette="Set2")
plt.title("Average Encounter Duration by Marital Status and Gender")
plt.xlabel("Marital Status")
plt.ylabel("Average Duration")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "MaritalStatusvsGendervsAverageEncounter")


In [81]:
filtered_encounter_patient_df.groupBy("marital_status", "gender", "age_group").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("marital_status", "gender", "age_group").show(truncate=False)

+--------------+------+-----------+------------------+
|marital_status|gender|age_group  |avg_duration      |
+--------------+------+-----------+------------------+
|Married       |female|Senior     |288.8011320754717 |
|Married       |male  |Middle Aged|60.76046511627907 |
|Never Married |female|Child      |15.0              |
|Never Married |female|Young Adult|23.32973484848485 |
|Never Married |male  |Young Adult|18.75945945945946 |
|Single        |female|Middle Aged|19.5              |
|Single        |female|Senior     |53.13312883435583 |
|Single        |male  |Middle Aged|28.023936170212767|
|Single        |male  |Senior     |227.02197802197801|
+--------------+------+-----------+------------------+



#### Marital Status, Gender, Age Group vs Average Encounter Duration

In [ ]:
marital_gender_age_avg_duration = filtered_encounter_patient_df.groupBy("marital_status", "gender", "age_group").agg(
    mean("encounter_duration").alias("avg_duration")
).orderBy("marital_status", "gender", "age_group").toPandas()

# Plotting FacetGrid to compare avg_duration for each combination of the categorical variables
sns.set(style="whitegrid")
g = sns.catplot(
    data=marital_gender_age_avg_duration, 
    x="marital_status", 
    y="avg_duration", 
    hue="gender", 
    col="age_group",  # Facet by age group
    kind="bar", 
    height=6, 
    aspect=1.2, 
    palette="Set2"
)
g.set_axis_labels("Marital Status", "Average Encounter Duration")
g.set_titles("Age Group: {col_name}")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "MaritalStatusGenderAgeGroupvsAverageEncounterDuration")

# Explanation of Benefits (Claims)

Explanation of benefits is the most important df since it contains and links claims, encounters and patient demographics. We can find out how much someone spent on their encounters. So we start by examining Explanation of benefits data frame, trying to clean and extract the payment information for encounters.

In [86]:
explanation_of_benefit_df.printSchema()
explanation_of_benefit_df.show(10, truncate=False)


root
 |-- eob_id: string (nullable = true)
 |-- identifier_claim_id: string (nullable = true)
 |-- identifier_claim_group: string (nullable = true)
 |-- status: string (nullable = true)
 |-- code: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- use: string (nullable = true)
 |-- patient_reference: string (nullable = true)
 |-- billable_period_start: string (nullable = true)
 |-- billable_period_end: string (nullable = true)
 |-- insurer_display: string (nullable = true)
 |-- provider_reference: string (nullable = true)
 |-- referral_reference: string (nullable = true)
 |-- claim_reference: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- care_team_provider_reference: string (nullable = true)
 |-- care_team_role: string (nullable = true)
 |-- insurance_coverage_reference: string (nullable = true)
 |-- insurance_coverage_display: string (nullable = true)
 |-- insurance_focal: boolean (nullable = true)
 |-- item_category_display: string (nu

In [87]:
explanation_of_benefit_df.select("eob_id").distinct().count()

887

We can see there are null values in columns because we might have nested data that we need to extract. 

In [88]:
explanation_of_benefit_df.select(
    mean("total_amount").alias("avg_total_amount"),
    mean("payment_amount").alias("avg_payment_amount"),
    sum("total_amount").alias("total_claimed"),
    sum("payment_amount").alias("total_paid")
).show()

+----------------+------------------+-------------+--------------------+
|avg_total_amount|avg_payment_amount|total_claimed|          total_paid|
+----------------+------------------+-------------+--------------------+
|            null|3650.2017721226207|         null|1.2622397728000022E7|
+----------------+------------------+-------------+--------------------+



In [ ]:
# explanation_of_benefit_df.select(
#     sum(col("total_amount").isNull().cast("int")).alias("null_total_amount"),
#     sum(col("payment_amount").isNull().cast("int")).alias("null_payment_amount")
# ).show()


In [90]:
explanation_of_benefit_df.select("total_amount", "payment_amount").show(truncate=False)

+------------+------------------+
|total_amount|payment_amount    |
+------------+------------------+
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |0.0               |
|null        |108.80000000000001|
|null        |108.80000000000001|
+------------+------------------+
only showing top 20 rows



In [91]:

explanation_of_benefit_df.filter(col("total_amount").isNotNull()).select("total_amount").show(truncate=False)


+------------+
|total_amount|
+------------+
+------------+



From the dataframe, adjudication is the column with many nested arrays that we need to extract values from since it contains all the co-pay, procedure amount. 

In [92]:
explanation_of_benefit_df.select("adjudication").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [94]:

explanation_of_benefit_df.select("adjudication").printSchema()


root
 |-- adjudication: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: struct (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- value: double (nullable = true)
 |    |    |-- category: struct (nullable = true)
 |    |    |    |-- coding: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |    |-- display: string (nullable = true)
 |    |    |    |    |    |-- system: string (nullable = true)



We created another dataframe to store the flattened adjudicated df but we still have nested columns here so we might have to extract a few more items before we join it with the encounter and patient df. 

In [95]:
from pyspark.sql.functions import explode

# Explode the adjudication array
adjudication_exploded_df = explanation_of_benefit_df.select(
    "eob_id",
    "patient_reference",
    explode("adjudication").alias("adjudication_item")
)


adjudication_exploded_df.show(truncate=False)


+------------------------------------+---------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|eob_id                              |patient_reference                            |adjudication_item                                                                                                                                                                                     |
+------------------------------------+---------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|03a8d3d0-079f-1ab2-eb4a-3961b3d71854|urn:uuid:7ca297d1-94e0-8069-4cdb-50f313de9a5f|{{USD, 27.200000000000003}, {[{https://bluebutton.cms.gov/resour

In [96]:
# Extract amount and category fields
adjudication_details_df = adjudication_exploded_df.select(
    "eob_id",
    "patient_reference",
    "adjudication_item.amount.value",
    "adjudication_item.amount.currency",
    "adjudication_item.category.coding"
)


adjudication_details_df.show(truncate=False)


+------------------------------------+---------------------------------------------+------------------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|eob_id                              |patient_reference                            |value             |currency|coding                                                                                                                                                                              |
+------------------------------------+---------------------------------------------+------------------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|03a8d3d0-079f-1ab2-eb4a-3961b3d71854|urn:uuid:7ca297d1-94e0-8069-4cdb-50f313de9a5f|27.200000000000003|USD     |[{http

In [97]:
adjudication_details_df.printSchema()

root
 |-- eob_id: string (nullable = true)
 |-- patient_reference: string (nullable = true)
 |-- value: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- coding: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- display: string (nullable = true)
 |    |    |-- system: string (nullable = true)



In [98]:
# Extract the first element of the 'coding' array and its fields
adjudication_details_df = adjudication_details_df.withColumn(
    "category_code",
    col("coding").getItem(0).getField("code")  
).withColumn(
    "category_display",
    col("coding").getItem(0).getField("display")  
)


adjudication_details_df.select("category_code", "category_display").show(truncate=False)


+-----------------------------------------------------------------------+-----------------------------------------+
|category_code                                                          |category_display                         |
+-----------------------------------------------------------------------+-----------------------------------------+
|https://bluebutton.cms.gov/resources/variables/line_coinsrnc_amt       |Line Beneficiary Coinsurance Amount      |
|https://bluebutton.cms.gov/resources/variables/line_prvdr_pmt_amt      |Line Provider Payment Amount             |
|https://bluebutton.cms.gov/resources/variables/line_sbmtd_chrg_amt     |Line Submitted Charge Amount             |
|https://bluebutton.cms.gov/resources/variables/line_alowd_chrg_amt     |Line Allowed Charge Amount               |
|https://bluebutton.cms.gov/resources/variables/line_bene_ptb_ddctbl_amt|Line Beneficiary Part B Deductible Amount|
|https://bluebutton.cms.gov/resources/variables/line_prcsg_ind_cd       

We join our new extracted adjudicated columns df with the original adjudicated df  

In [99]:
adjudication_with_claims_df = adjudication_details_df.join(
    explanation_of_benefit_df.select("eob_id", "insurance_coverage_display", "billable_period_start"),
    ["eob_id"],
    how="inner"
)

adjudication_with_claims_df.show(truncate=False)

+------------------------------------+---------------------------------------------+------------------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+-----------------------------------------+--------------------------+-------------------------+
|eob_id                              |patient_reference                            |value             |currency|coding                                                                                                                                                                              |category_code                                                          |category_display                         |insurance_coverage_display|billable_period_start    |
+------------------------------------+----------------------------------------

In [100]:
adjudication_with_claims_cleaned_df = adjudication_with_claims_df.drop("coding", "category_code", "insurance_coverage_display")
adjudication_with_claims_cleaned_df.show(truncate=False)
adjudication_with_claims_cleaned_df.printSchema()

+------------------------------------+---------------------------------------------+------------------+--------+-----------------------------------------+-------------------------+
|eob_id                              |patient_reference                            |value             |currency|category_display                         |billable_period_start    |
+------------------------------------+---------------------------------------------+------------------+--------+-----------------------------------------+-------------------------+
|03a8d3d0-079f-1ab2-eb4a-3961b3d71854|urn:uuid:7ca297d1-94e0-8069-4cdb-50f313de9a5f|27.200000000000003|USD     |Line Beneficiary Coinsurance Amount      |2005-05-02T03:59:33-04:00|
|03a8d3d0-079f-1ab2-eb4a-3961b3d71854|urn:uuid:7ca297d1-94e0-8069-4cdb-50f313de9a5f|27.200000000000003|USD     |Line Beneficiary Coinsurance Amount      |2005-05-02T03:59:33-04:00|
|03a8d3d0-079f-1ab2-eb4a-3961b3d71854|urn:uuid:7ca297d1-94e0-8069-4cdb-50f313de9a5f|27.20000000

Creating our cleaned and final df for claims also cleaning the patient reference so we can match that with the encounter_patient df.

In [102]:
eob_final_df_cleaned = adjudication_with_claims_cleaned_df.withColumn(
    "patient_reference",
    regexp_replace("patient_reference", "urn:uuid:", "")
)

eob_final_df_cleaned.select("patient_reference").distinct().show(truncate=False)
eob_final_df_cleaned.show(10, truncate=False)

+------------------------------------+
|patient_reference                   |
+------------------------------------+
|7ca297d1-94e0-8069-4cdb-50f313de9a5f|
|b4b67625-c44c-a832-7aba-3a2ae02a4ef1|
|0ff36c90-7124-3569-9591-75c6386f45f5|
|c8af2e36-e7d2-fc26-de8c-8936c9e4705e|
|43dceade-79e7-fc0f-8245-59ec5feb4174|
|3ed96c8a-2671-2959-57cd-b3aeb8b16af0|
|37b0788d-2389-b706-52e0-7938f80f3f23|
|8fc4d8c6-1ae5-2736-1158-ef55c405db8d|
|5bbda3f6-ee30-6a51-ebbd-19cd0a6b119d|
|33c6c178-b195-c16c-0045-02f5f4b53073|
|d59c9ad1-8a83-f0f0-a080-635ff42b43fa|
|fb47aa81-57a0-edc1-185d-891ad133034b|
|fbcfe59a-ee76-74ed-240c-a0f3c9d46c04|
|000002d9-3788-19b0-0631-c4cad43fe513|
|047c5aaf-d8cb-a31b-189a-f28e0efb78e0|
+------------------------------------+

+------------------------------------+------------------------------------+------------------+--------+-----------------------------------+-------------------------+
|eob_id                              |patient_reference                   |value            

Joining this df with our patient and encounter df to get an explanation of benefits with demographics and encounter information

In [114]:
eob_with_demographics_df = eob_final_df_cleaned.join(
    filtered_encounter_patient_df.select("patient_reference", "gender", "age", "marital_status", "age_group"),
    ["patient_reference"],
    how="inner"
)

eob_with_demographics_df.printSchema()
eob_with_demographics_df.show(truncate=False)


root
 |-- patient_reference: string (nullable = true)
 |-- eob_id: string (nullable = true)
 |-- value: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- category_display: string (nullable = true)
 |-- billable_period_start: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = false)
 |-- age_group: string (nullable = false)

+------------------------------------+------------------------------------+------------------+--------+-----------------------------------+-------------------------+------+---+--------------+---------+
|patient_reference                   |eob_id                              |value             |currency|category_display                   |billable_period_start    |gender|age|marital_status|age_group|
+------------------------------------+------------------------------------+------------------+--------+-----------------------------------+-----------------------

In [115]:
eob_with_demographics_df.select("category_display").distinct().show(truncate=False)


+-----------------------------------------+
|category_display                         |
+-----------------------------------------+
|Line Submitted Charge Amount             |
|Line Beneficiary Coinsurance Amount      |
|Line Allowed Charge Amount               |
|Line Provider Payment Amount             |
|Line Beneficiary Part B Deductible Amount|
|Line Processing Indicator Code           |
+-----------------------------------------+



Standardizing the categories 

In [116]:
eob_with_demographics_df = eob_with_demographics_df.withColumn(
    "category_display_normalized",
    trim(lower(col("category_display")))
)

In [117]:
eob_with_demographics_df.select("category_display", "category_display_normalized").distinct().show(truncate=False)

+-----------------------------------------+-----------------------------------------+
|category_display                         |category_display_normalized              |
+-----------------------------------------+-----------------------------------------+
|Line Provider Payment Amount             |line provider payment amount             |
|Line Processing Indicator Code           |line processing indicator code           |
|Line Submitted Charge Amount             |line submitted charge amount             |
|Line Beneficiary Part B Deductible Amount|line beneficiary part b deductible amount|
|Line Allowed Charge Amount               |line allowed charge amount               |
|Line Beneficiary Coinsurance Amount      |line beneficiary coinsurance amount      |
+-----------------------------------------+-----------------------------------------+



In [118]:
eob_with_demographics_df = eob_with_demographics_df.withColumn(
    "category_display",
    trim(lower(col("category_display")))
)

eob_with_demographics_df = eob_with_demographics_df.withColumn(
    "category_display",
    when(col("category_display") == "line submitted charge amount", "Submitted Charge")
    .when(col("category_display") == "line beneficiary coinsurance amount", "Coinsurance Amount")
    .when(col("category_display") == "line allowed charge amount", "Allowed Charge")
    .when(col("category_display") == "line provider payment amount", "Provider Payment")
    .when(col("category_display") == "line beneficiary part b deductible amount", "Part B Deductible")
    .when(col("category_display") == "line processing indicator Code", "Processing Indicator")
    .otherwise("Unknown")  # if I have any unexpected values just in case
)

eob_with_demographics_df.select("category_display").distinct().show(truncate=False)

+------------------+
|category_display  |
+------------------+
|Part B Deductible |
|Unknown           |
|Provider Payment  |
|Submitted Charge  |
|Allowed Charge    |
|Coinsurance Amount|
+------------------+



Removed the unknown and part b deductible from my categories since unknown categories have no values and part b deductible is always 0

In [149]:
eob_with_demographics_filtered_df = eob_with_demographics_df.filter(
    (col("category_display") != "Unknown") & 
    (col("category_display") != "Part B Deductible")
)
eob_with_demographics_filtered_df.groupBy("category_display").agg(
    mean("value").alias("avg_value")
).show(truncate=False)

+------------------+------------------+
|category_display  |avg_value         |
+------------------+------------------+
|Provider Payment  |2316.1156098525403|
|Submitted Charge  |2895.1445123169983|
|Allowed Charge    |2895.1445123169983|
|Coinsurance Amount|579.0289024631351 |
+------------------+------------------+



#### Average Value by Category

In [ ]:
category_display_avg_value = eob_with_demographics_filtered_df.groupBy("category_display").agg(
    mean("value").alias("avg_value")
).toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=category_display_avg_value, x="category_display", y="avg_value", palette="Set2")
plt.title("Average Value by Category Display")
plt.xlabel("Category Display")
plt.ylabel("Average Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

upload_plot_to_s3(plt, "AverageValuebyCategory")

In [133]:
eob_with_demographics_filtered_df.groupBy("category_display", "gender").agg(
    mean("value").alias("avg_value")
).orderBy("category_display", "gender").show(truncate=False)

+------------------+------+------------------+
|category_display  |gender|avg_value         |
+------------------+------+------------------+
|Allowed Charge    |female|2782.4680875002423|
|Allowed Charge    |male  |3215.473803239008 |
|Coinsurance Amount|female|556.4936174997423 |
|Coinsurance Amount|male  |643.0947606466993 |
|Part B Deductible |female|0.0               |
|Part B Deductible |male  |0.0               |
|Provider Payment  |female|2225.974469998969 |
|Provider Payment  |male  |2572.3790425867974|
|Submitted Charge  |female|2782.4680875002423|
|Submitted Charge  |male  |3215.473803239008 |
+------------------+------+------------------+



#### Average Value by Category Display and Gender

In [152]:
category_gender_avg_value = eob_with_demographics_filtered_df.groupBy("category_display", "gender").agg(
    mean("value").alias("avg_value")
).orderBy("category_display", "gender").toPandas()

plt.figure(figsize=(12, 8))
sns.catplot(data=category_gender_avg_value, x="category_display", y="avg_value", hue="gender", kind="bar", height=6, aspect=2)
plt.title("Average Value by Category Display and Gender")
plt.xlabel("Category Display")
plt.ylabel("Average Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "AverageValuebyCategoryDisplayandGender")

NameError: name 'category_gender_avg_value' is not defined

<Figure size 1200x800 with 0 Axes>

In [134]:
eob_with_demographics_filtered_df.groupBy("category_display", "age_group").agg(
    mean("value").alias("avg_value")
).orderBy("category_display", "age_group").show(truncate=False)


+------------------+-----------+------------------+
|category_display  |age_group  |avg_value         |
+------------------+-----------+------------------+
|Allowed Charge    |Child      |168.5460000000002 |
|Allowed Charge    |Middle Aged|5557.485991732535 |
|Allowed Charge    |Senior     |2572.248099286143 |
|Allowed Charge    |Young Adult|2154.4368514531943|
|Coinsurance Amount|Child      |33.70920000000194 |
|Coinsurance Amount|Middle Aged|1111.4971983471346|
|Coinsurance Amount|Senior     |514.4496198569227 |
|Coinsurance Amount|Young Adult|430.88737029048247|
|Part B Deductible |Child      |0.0               |
|Part B Deductible |Middle Aged|0.0               |
|Part B Deductible |Senior     |0.0               |
|Part B Deductible |Young Adult|0.0               |
|Provider Payment  |Child      |134.83680000000777|
|Provider Payment  |Middle Aged|4445.988793388538 |
|Provider Payment  |Senior     |2057.798479427691 |
|Provider Payment  |Young Adult|1723.5494811619299|
|Submitted C

#### Avg Value by Category and Age Group

In [ ]:
category_age_group_avg_value = eob_with_demographics_filtered_df.groupBy("category_display", "age_group").agg(
    mean("value").alias("avg_value")
).orderBy("category_display", "age_group").toPandas()

plt.figure(figsize=(12, 8))
sns.catplot(data=category_age_group_avg_value, x="category_display", y="avg_value", hue="age_group", kind="bar", height=6, aspect=2)
plt.title("Average Value by Category Display and Gender")
plt.xlabel("Category Display")
plt.ylabel("Average Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "AvgValuebyCategoryandAgeGroup")

#### Average Value by Category Display and Age Group

In [ ]:
plt.figure(figsize=(12, 8))
sns.catplot(data=category_age_group_avg_value, x="category_display", y="avg_value", hue="age_group", kind="bar", height=6, aspect=2)
plt.title("Average Value by Category Display and Age Group")
plt.xlabel("Category Display")
plt.ylabel("Average Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "AverageValuebyCategoryDisplayandAgeGroup")

In [137]:
eob_with_demographics_filtered_df.groupBy("category_display", "gender", "age_group", "marital_status").agg(
    mean("value").alias("avg_value")
).orderBy("category_display", "gender", "age_group", "marital_status").show(truncate=False)

category_gender_age_group_marital_avg_value = eob_with_demographics_filtered_df.groupBy(
    "category_display", "gender", "age_group", "marital_status"
).agg(
    mean("value").alias("avg_value")
).orderBy("category_display", "gender", "age_group", "marital_status").toPandas()


+------------------+------+-----------+--------------+------------------+
|category_display  |gender|age_group  |marital_status|avg_value         |
+------------------+------+-----------+--------------+------------------+
|Allowed Charge    |female|Child      |Never Married |168.5460000000002 |
|Allowed Charge    |female|Middle Aged|Single        |2060.843488372074 |
|Allowed Charge    |female|Senior     |Married       |3275.954480280915 |
|Allowed Charge    |female|Senior     |Single        |1915.7266570612328|
|Allowed Charge    |female|Young Adult|Never Married |2770.2783545350862|
|Allowed Charge    |male  |Middle Aged|Married       |2910.8244898815797|
|Allowed Charge    |male  |Middle Aged|Single        |8194.897974682872 |
|Allowed Charge    |male  |Senior     |Single        |1253.1209677639945|
|Allowed Charge    |male  |Young Adult|Never Married |409.2140056550415 |
|Coinsurance Amount|female|Child      |Never Married |33.70920000000194 |
|Coinsurance Amount|female|Middle Aged

In [ ]:
plt.figure(figsize=(16, 12))
sns.catplot(
    data=category_gender_age_group_marital_avg_value, 
    x="category_display", 
    y="avg_value", 
    hue="gender", 
    col="age_group", 
    row="marital_status", 
    kind="bar", 
    height=5, 
    aspect=1.5
)
plt.suptitle("Average Value by Category Display, Gender, Age Group, and Marital Status", size=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)  
plt.show()
upload_plot_to_s3(plt, "AverageValuebyCategoryDisplayandAgeGroupCatplot")

In [ ]:
pivoted_data = category_gender_age_group_marital_avg_value.pivot_table(
    values="avg_value", 
    index=["gender", "age_group", "marital_status"], 
    columns="category_display"
)

plt.figure(figsize=(14, 10))
sns.heatmap(pivoted_data, annot=True, cmap="YlGnBu", fmt=".2f", linewidths=0.5)
plt.title("Heatmap of Average Value by Gender, Age Group, Marital Status, and Category Display")
plt.xlabel("Category Display")
plt.ylabel("Gender, Age Group, Marital Status")
plt.tight_layout()
plt.show()
upload_plot_to_s3(plt, "AverageValuebyCategoryDisplayandAgeGroupHeatmap")